# Run Instructions

Press `space` to progress the slide show once it's started.

- Click on the cell below and run (`shift` + `enter`) to import dependencies and make functions accessible.
- Start the slide show
    - Click on the "Enter/Exit RISE Slideshow" (graph icon in the toolbar above)
    - Or, press `Alt` + `r`

In [1]:
from matplotlib import rc
import time
import random
from IPython import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
from scipy.stats import binned_statistic as binsta
from scipy.special import logsumexp
import palettable as pal
clrx = pal.cartocolors.qualitative.Prism_10.mpl_colors
clr = tuple(x for n,x in enumerate(clrx) if n in [1,2,4,5,6])
clr2 = pal.cartocolors.sequential.agSunset_7.mpl_colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

def p_overlap(na,nb,nab,pool=60):
    p_s = np.zeros(pool+1)
    # reference: hypergeom.pmf(outcome, Total, hits, Draws, loc=0)
    for s in np.arange(pool+1):
        # p_sa is the probability that we'd get sa from the overlap (s), just in na draws of a
        p_sa = hypergeom.pmf(np.arange(pool+1),pool,s,na)
        # p_nab_given_sa is the probability of getting that nab, given sa
        p_nab_given_sa = hypergeom.pmf(nab,pool,np.arange(pool+1),nb)
        p_s[s] = np.dot(p_sa,p_nab_given_sa)
    return p_s/np.sum(p_s)


def credible_interval(na,nb,nab,pct=90,pool=60):
    p_s = p_overlap(na,nb,nab,pool=pool)
    cdf = np.cumsum(p_s)
    ccdf = np.flipud(np.cumsum(np.flipud(p_s)))
    # adjust for fractions vs percents; put everything as a fraction
    if pct > 1:
        pct = pct/100
    cutoff = (1-pct)/2
    # get the lower bound. 
    # it's the first index at which cdf ≥ cutoff
    try:
        lower = np.where(cdf >= cutoff)[0][0]
    except IndexError:
        lower = 0
    # get the upper bound
    # it's the first index at which ccdf ≥ 0.05
    try:
        upper = np.where(ccdf >= cutoff)[0][-1]
    except IndexError:
        upper=pool
    expectation = np.dot(np.arange(pool+1),p_s)
    # Sanity and indexing check: uncomment this line to see true tail probability ≤ 0.05
    # print([cdf[lower-1],(1-ccdf[upper+1])])
    return lower,expectation,upper


# Bayes-optimal estimation of overlap between populations of fixed size
Daniel B. Larremore 

Atreca journal club<br>
John Vivian<br>
December 9th, 2019

## Motivating Question

<center>How we can we more accurately estimate the population overlap of antibodies, or antibody lineages, between donors when sampling?</center>

## The Problem

![fig1](files/figures/fig1.png)

## Existing methods

Assuming $n_a$ and $n_b$ are samples from two populations with an intersection of $n_{ab}$

### Jaccard index

$$\mathring{J} = \frac{n_{ab}}{n_a + n_b - n_{ab}} = \frac{n_a \cap n_b}{n_a \cup n_b}$$

### Sørenson-Dice coefficient

$$\mathring{S} = \frac{n_{ab}}{\frac{1}{2}(n_a + n_b)}$$

<center>Intuitively, when two populations are identical these equal 1 and when they are entirely independent they equal 0.</center>

## What is the problem with these methods?

<br>
<br>
<br>
<center>Sampling</center>

## Problems with sampling

Imagine...<br>
$n_a = n_b = 10$<br>
$n_{ab} = 5$

With perfect sampling...<br>
$\mathring{J} = \frac{1}{3}$<br>
$\mathring{S} = \frac{1}{2}$<br>

<center>Yet, if we only sample 9 out of 10 species drawn from these populations, the expected values average $E[\mathring{J}] = 0.29$ and $E[\mathring{S}] = 0.45$, representing biases of $-12\%$ and $-10\%$.</center>

In [ ]:
a, b = range(10), range(5, 15)
avgs, roll_avg = [], []
for i in range(125):
    na, nb = set(random.sample(a, 9)), random.sample(b, 9)
    nab = na.intersection(nb)
    avgs.append(len(nab) /  9)
    roll_avg.append(np.mean(avgs))
    plt.figure(figsize=(16, 4))
    plt.plot(range(i + 1), roll_avg, label='Average') 
    plt.axhline(0.45, c='r', ls='--', label='Asymptote')
    plt.axhline(0.5, c='g', label='True value')
    plt.xlabel('Trials'); plt.ylabel('Sørenson-Dice coefficient')
    plt.title('Simulating the expected value of the Sørenson-Dice coefficient')
    plt.legend(loc='lower center')
    display   .display(plt.gcf()); display.clear_output(wait=True); plt.close()

# Bayesian Repertoire Overlap

## Goals

1. Accurately estimate the true repertoire overlap $s$ given the knowledge that $n_a$ samples from group $a$ and $n_b$ samples from group $b$ share $n_{ab}$ types.

2. Given that the sampling of $n_a$ and $n_b$ is stochastic, we should ideally quantify the uncertainty of the estimate.

## Sidebar: The hypergeometric distribution

### Relationship to other distributions (conceptually)

1. Bernoulli distribution: $f(k;p) = p^k(1-p)^{1-k}$ where $k \in \{0, 1\}$
    1. Ex: Outcome of a single coin flip

2. Binomial distribution: $Pr(X=k) = {n \choose k} p^kq^{n-k}$
    1. Ex: An independent series of coin flips. E.g. Getting 5 heads in 15 coin flips. 
    2. Where ${n \choose k} = \frac{n!}{k!(n-k)!}$

### Relationship to other distributions (continued)
2. Binomial distribution: $Pr(X=k) = {n \choose k} p^kq^{n-k}$
    1. Ex: An independent series of coin flips. E.g. Getting 5 heads in 15 coin flips. 
    2. Where ${n \choose k} = \frac{n!}{k!(n-k)!}$

3. Hypergeometric Distribution: $Pr(X=k) = \frac{{s \choose k}{N-s \choose n-k}}{{N \choose n}}$
    1. Ex: Choosing 4 red balls out of a bucket of 10 balls in which 5 are red. Just like the binomial distribution, but _without replacement_, meaning the selections are not independent. 

## Building intuition around the method

Definition: Given $s$ special objects among a total of $N$ wherein $n$ objects are drawn uniformly at random without replacement. We can write this as $\mathcal{H}(s, N, n)$

Considering sampling $n_a$ genes from parasite $a$ with 60 total genes of interest. Of the 60, suppose $s$ are interesting because they are shared by parasite $b$. We can represent the number of shared sequences captured by sequencing parasite $a$ by the random variable $S_a = \mathcal{H}(s, 60, n_a)$. 

If we now consider sampling $n_b$ genes from parasite $b$'s 60 genes, in which $s_a$ are special because they are shared by both parasites and appeared when parasite $a$ was sequenced. This is identical to the process of sampling parasite $a$, but with $s_a$ special sequences instead of $s$, which gives us $\mathcal{H}(s_a, 60, n_b)$.

We can therefore substitute the random variable $S_a$ for the fixed value $s_a$. Replacing a fixed parameter with another random variable is the hallmark of hierarchical Bayesian models. Ergo, the probability of a particular number of shared sequences in the samples $n_{ab}$ is given by:

$$P(n_{ab} \mid n_a, n_b, s) \sim \mathcal{H}(\mathcal{H}(s, 60, n_a), 60, n_b)$$

## Bayes' Rule

We did it!... almost. This formula gives us the probability of getting $n_{ab}$ samples and we want to go the other way and calculate the probability of $s$, true number of special objects, given the other values. Fortunately, we can do this readily by applying Bayes' rule, which is a method for inverting conditional probabilities.

$$P(s \mid n_a, n_b, n_{ab}) = \frac{P(n_{ab} \mid n_a, n_b, s)P(s)}{P(n_{ab})}$$

<center>Conceptually, this is the entire method. Calculating equation 6 in the paper is just boring algebra.</center>

## Comparing methods

In [ ]:
na = 47       # 47 samples from group A
nb = 32       # 32 samples from group B
nab = 20      # 20 of the samples are shared between A and B
pool = 60     # There are a total of 60 items in both A and B
# s = ?         What is the expected value of s in the population?

In [ ]:
pts = pool*2*nab/(na+nb)
ps = p_overlap(na,nb,nab,pool=pool)
lower,shat,upper = credible_interval(na,nb,nab,pool=pool)
x = np.arange(lower,upper+1)
y = np.copy(ps[x])
x = np.append(x,upper)
y = np.append(y,0)
x = np.insert(x,0,lower)
y = np.insert(y,0,0)
er = np.zeros([2,1])
er[0] = shat-lower
er[1] = upper-shat

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(np.arange(pool+1), ps, label='posterior distribution', linestyle='-', marker='o', ms=5, color = clr[2])
plt.scatter(shat,0, facecolor=[1,1,1], edgecolor=clr[2], linewidth = 2, s=90, marker='o', label='Bayesian estimate', clip_on=False,zorder=100)
plt.fill(x, y, facecolor=clr[2], edgecolor=[0,0,0], alpha=0.2, label='90% credible interval',)
plt.scatter(pts,0, linewidth=2, color='k', marker='x', s=90, label='Sorenson-Dice coefficient, S', clip_on=False,zorder=100)
plt.ylim(bottom=0)
plt.legend()
plt.ylabel('Posterior probability P(s)')
plt.xlabel('s');

## Sørenson-Dice coefficient routinely under-estimate the true $s$

![fig3](files/figures/fig3.png)

![fig4](files/figures/fig4.png)

# Quantification of uncertainty, another benefit

![fig6](files/figures/fig6.png)

## Application to antibody intersection

<br>
<center>Out of the box, what issue with this method prevents us from estimating the true antibody lineage overlap between donors?</center>

<br>
<br>
<center>The answer is in the paper title: <em>populations of fixed size</em></center>

The author already provides a generalization that works when the populations $N_a$ and $N_b$ are different, but they are still a fixed size.

What are some possible solutions?

## Extending the model

One thought was to extend the model by replacing the fixed population parameters $N_a$ and $N_b$ with distributions. This assumes that we have a way of getting an average of the "total" plasmablast population across numerous individuals. How would we do this? How large would the uncertainty be? Could we use different distributions for those with an acute infection versus background?

![bgm1](files/figures/bgm1.png)

![bgm1](files/figures/bgm2.png)

<center>Thank you</center>